In [1]:
import pandas as pd
import numpy as np
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
import os
import logging
import time
import matplotlib.pyplot as plt  # <--- THÊM THƯ VIỆN VẼ BIỂU ĐỒ

logging.getLogger('cmdstanpy').setLevel(logging.ERROR)
warnings.filterwarnings("ignore")
start_time_total = time.time()

# --- CẤU HÌNH LƯU ẢNH ---
plot_folder = 'forecast_plots'
if not os.path.exists(plot_folder):
    os.makedirs(plot_folder)
    print(f"Đã tạo thư mục lưu biểu đồ: {plot_folder}")

Đã tạo thư mục lưu biểu đồ: forecast_plots


In [2]:
# --- 1. CẤU HÌNH CÁC NHÓM (GROUPS) ---
family_groups = {
    "Ultra Low": ["HOME APPLIANCES", "BABY CARE", "BOOKS"],
    "Low": [
        "LINGERIE", "CELEBRATION", "PLAYERS AND ELECTRONICS", "AUTOMOTIVE",
        "LADIESWEAR", "PET SUPPLIES", "LAWN AND GARDEN", "BEAUTY",
        "SCHOOL AND OFFICE SUPPLIES", "MAGAZINES", "HARDWARE",
    ],
    "Medium": [
        "PREPARED FOODS", "LIQUOR,WINE,BEER","HOME AND KITCHEN I",
        "GROCERY II", "SEAFOOD", "HOME AND KITCHEN II"
    ],
    "High": [
        "BREAD/BAKERY", "POULTRY", "PERSONAL CARE", "MEATS",
        "DELI", "EGGS", "HOME CARE", "FROZEN FOODS", "GROCERY I",
        "BEVERAGES" , "PRODUCE", "CLEANING", "DAIRY"
    ]
}

n_test_days = 90
filename = 'final_dataset.csv'


In [3]:
# --- 2. HÀM TÍNH ACCURACY ---
def calculate_forecast_accuracy(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    sum_abs_errors = np.sum(np.abs(y_true - y_pred))
    sum_abs_true = np.sum(np.abs(y_true))
    if sum_abs_true == 0:
        return 1.0 if sum_abs_errors == 0 else 0.0
    return 1.0 - (sum_abs_errors / sum_abs_true)

In [4]:
# --- 3. NẠP DỮ LIỆU ---
try:
    df_master = pd.read_csv(filename)
    df_master['date'] = pd.to_datetime(df_master['date'])
    df_master = df_master.set_index('date').sort_index()
    print(f"Đã nạp dữ liệu thành công: {len(df_master)} dòng.")
except FileNotFoundError:
    raise SystemExit(f"Lỗi: Không tìm thấy file '{filename}'")

Đã nạp dữ liệu thành công: 444576 dòng.


In [ ]:
# --- 4. VÒNG LẶP CHÍNH (XỬ LÝ & LƯU FILE CON) ---
print("\n" + "="*60)
print("PHẦN 1: CHẠY SARIMAX (AUTO_ARIMA) VÀ VẼ BIỂU ĐỒ")
print("="*60)

generated_files = [] 

for group_name, families in family_groups.items():
    print(f"\n>>> Đang xử lý nhóm: {group_name}")
    
    group_results = []
    
    for family in families:
        print(f"  Processing: {family}...", end=" ")
        
        # --- 4a. Chuẩn bị dữ liệu ---
        df_family_raw = df_master[df_master['family'] == family]
        
        if df_family_raw.empty:
            print("[Bỏ qua: Không có dữ liệu]")
            continue
            
        # Tổng hợp
        df_family = df_family_raw.groupby('date').agg(
            sales=('sales', 'sum'),
            onpromotion=('onpromotion', 'sum'),
            is_holiday=('is_holiday', 'max'),
            dcoilwtico=('dcoilwtico', 'mean')
        )
        
        # Đảm bảo tần suất ngày & Fillna
        df_family = df_family.asfreq('D')
        df_family['sales'] = df_family['sales'].fillna(0)
        df_family['onpromotion'] = df_family['onpromotion'].fillna(0)
        df_family['is_holiday'] = df_family['is_holiday'].fillna(0)
        df_family['dcoilwtico'] = df_family['dcoilwtico'].interpolate(limit_direction='both').fillna(0)
        
        if len(df_family) <= n_test_days:
            print("[Bỏ qua: Dữ liệu quá ngắn]")
            continue
            
        y = df_family['sales']
        exog = df_family[['onpromotion', 'is_holiday', 'dcoilwtico']]
        
        # Chia dữ liệu
        y_train_search = y.iloc[:-n_test_days]
        exog_train_search = exog[['onpromotion', 'is_holiday']].iloc[:-n_test_days]
        
        y_train = y.iloc[:-n_test_days]
        y_test = y.iloc[-n_test_days:]
        exog_train = exog.iloc[:-n_test_days]
        exog_test = exog.iloc[-n_test_days:]
        
        # --- 4b. Chạy AUTO_ARIMA ---
        try:
            auto_model = pm.auto_arima(
                y_train_search,
                exogenous=exog_train_search,
                m=7,                
                seasonal=True,      
                d=None, D=None,      
                max_p=3, max_q=3,    
                max_P=2, max_Q=2,
                n_jobs=1,            
                stepwise=True,      
                suppress_warnings=True,
                error_action='ignore',
                trace=False
            )
            best_order = auto_model.order
            best_seasonal_order = auto_model.seasonal_order
            print(f"[{best_order}{best_seasonal_order}]", end=" ")
            
        except Exception as e:
            print(f"[AutoARIMA Fail -> Default]", end=" ")
            best_order = (1, 1, 1)
            best_seasonal_order = (0, 1, 1, 7)

        # --- 4c. Huấn luyện SARIMAX & Dự báo ---
        try:
            model = SARIMAX(y_train,
                            exog=exog_train,
                            order=best_order,
                            seasonal_order=best_seasonal_order,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
            
            sar = model.fit(disp=False, low_memory=True)
            
            # Lấy kết quả dự báo và khoảng tin cậy (Confidence Interval)
            forecast_res = sar.get_forecast(steps=len(y_test), exog=exog_test)
            y_pred = forecast_res.predicted_mean
            y_pred = y_pred.clip(lower=0)
            conf_int = forecast_res.conf_int() # Lấy khoảng tin cậy
            
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            acc = calculate_forecast_accuracy(y_test, y_pred) * 100
            mean_true = np.mean(y_test)
            if mean_true != 0:
                wape = (mae / mean_true)
            else:
                wape = 0.0
            print(f"-> R2={r2:.2f}")
            
            # --- 4d. VẼ BIỂU ĐỒ & LƯU ẢNH (NEW) ---
            try:
                plt.figure(figsize=(12, 6))
                
                # Chỉ vẽ 180 ngày cuối của tập train để biểu đồ thoáng
                train_subset = y_train.iloc[-180:] 
                
                plt.plot(train_subset.index, train_subset, label='Train (Last 6 months)', color='gray', alpha=0.6)
                plt.plot(y_test.index, y_test, label='Actual (Test)', color='blue', linewidth=2)
                plt.plot(y_pred.index, y_pred, label='Forecast', color='red', linestyle='--', linewidth=2)
                
                # Vẽ vùng tin cậy (Confidence Interval)
                plt.fill_between(conf_int.index, 
                                 conf_int.iloc[:, 0].clip(lower=0), 
                                 conf_int.iloc[:, 1], 
                                 color='pink', alpha=0.3, label='95% Conf. Int.')
                
                plt.title(f"Forecast: {family} | R2: {r2:.2f} | RMSE: {rmse:.2f} | Acc: {acc:.1f}%")
                plt.legend(loc='upper left')
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                
                # Lưu file
                safe_family_name = family.replace("/", "_").replace(" ", "_") # Xử lý tên file
                plot_filename = f"{plot_folder}/{group_name}_{safe_family_name}.png"
                plt.savefig(plot_filename)
                plt.close() # Đóng figure để giải phóng RAM
                
            except Exception as plot_error:
                print(f" [Lỗi vẽ hình: {plot_error}]", end=" ")

            # --- Lưu kết quả vào list ---
            group_results.append({
                'family': family,
                'RMSE': rmse,
                'MAE': mae,
                'R2': r2,
                'Accuracy_Pct': acc,
                'WAPE': wape,
                'Order': str(best_order),
                'Seasonal_Order': str(best_seasonal_order)
            })
            
        except Exception as e:
            print(f" -> [Lỗi Fit: {str(e)[:20]}]")

    # --- 4e. Lưu File CSV Nhóm ---
    if group_results:
        df_group = pd.DataFrame(group_results)
        avg_metrics = df_group[['RMSE', 'MAE', 'R2', 'Accuracy_Pct']].mean()
        avg_row = avg_metrics.to_dict()
        avg_row['family'] = 'AVERAGE (Trung bình nhóm)'
        avg_row['Order'] = '-'
        avg_row['Seasonal_Order'] = '-'
        
        df_group_final = pd.concat([df_group, pd.DataFrame([avg_row])], ignore_index=True)
        
        group_filename = f"SARIMAX_Auto_{group_name}_results.csv"
        df_group_final.to_csv(group_filename, index=False, float_format='%.4f')
        generated_files.append((group_name, group_filename)) 
        print(f"  >> Đã lưu file nhóm: {group_filename}")


PHẦN 1: CHẠY SARIMAX (AUTO_ARIMA) VÀ VẼ BIỂU ĐỒ

>>> Đang xử lý nhóm: Ultra Low
  Processing: HOME APPLIANCES... [(2, 1, 2)(1, 0, 1, 7)] -> R2=-0.64
  Processing: BABY CARE... [(3, 1, 3)(2, 0, 1, 7)] -> R2=-0.07
  Processing: BOOKS... [(0, 1, 3)(1, 0, 1, 7)] -> R2=-27.60
  >> Đã lưu file nhóm: SARIMAX_Auto_Ultra Low_results.csv

>>> Đang xử lý nhóm: Low
  Processing: LINGERIE... [(1, 1, 1)(1, 0, 2, 7)] -> R2=-0.18
  Processing: CELEBRATION... 

In [ ]:


# --- 5. GỘP FILE (MERGE) ---
print("\n" + "="*60)
print("PHẦN 2: GỘP TẤT CẢ THÀNH FILE TỔNG HỢP")
print("="*60)

all_dataframes = []

if not generated_files:
    print("Không có file nào được tạo ra để gộp.")
else:
    for group_name, filename in generated_files:
        if os.path.exists(filename):
            try:
                df = pd.read_csv(filename)
                df.insert(0, 'Group_Category', group_name)
                all_dataframes.append(df)
                print(f"  [OK] Đã đọc để gộp: {filename}")
            except Exception as e:
                print(f"  [Lỗi] Không đọc được {filename}: {e}")

    if all_dataframes:
        final_df = pd.concat(all_dataframes, ignore_index=True)
        final_output_filename = "SARIMAX_RESULTS.csv"
        final_df.to_csv(final_output_filename, index=False, float_format='%.4f')
        
        total_time = (time.time() - start_time_total) / 60
        print("\n" + "="*60)
        print(f"HOÀN TẤT! Tổng thời gian: {total_time:.2f} phút")
        print(f"File kết quả: {final_output_filename}")
        print(f"Biểu đồ đã lưu trong thư mục: {plot_folder}/")
        print("="*60)
        print("\n5 dòng đầu của file tổng hợp:")
        print(final_df.head().to_string())
    else:
        print("Không tạo được DataFrame tổng hợp.")